In [4]:
import json
import os
from flask import Flask, render_template, request
from flask_cors import CORS
# from helpers.similarity import get_title_sim, cossim_scores
from helpers import preprocessing, similarity
import pandas as pd
from collections import defaultdict

In [5]:

with open("init.json", 'r') as file:
    data = json.load(file)
    kdramas_df = pd.DataFrame(data)


In [ ]:
def cossim_search(query):
    inv_idx = preprocessing.build_inverted_index(kdramas_df['synopsis'])
    idf_dict = preprocessing.compute_idf(inv_idx, len(kdramas_df['synopsis']))
    doc_norms = preprocessing.compute_doc_norms(inv_idx, idf_dict, len(kdramas_df['synopsis']))
    scores = similarity.index_search(query, inv_idx, idf_dict, doc_norms)
    doc_ids = [doc_id for _, doc_id in scores]
    ranked_docs = kdramas_df.iloc[doc_ids]
    matches_filtered = ranked_docs if not ranked_docs.empty else kdramas_df
    matches_filtered_json = matches_filtered.to_json(orient='records')
    return [(score, kdramas_df.iloc[id]["name"]) for score, id in scores]

cossim_search("squid")

[(9, 1), (152, 1), (168, 1), (195, 1), (266, 1), (290, 1), (507, 1), (697, 2), (717, 2), (784, 1), (1232, 2), (1457, 1), (1511, 1), (1570, 1), (1617, 1)]
squid


[]